Example
=======

As a first example, we use an image dataset and a pre-trained model to classify the images. We use the `ReVel` framework to load the dataset and the model, and to perform the classification. We also added the `procedures` module to help us with the classification process.

In [5]:
import torch
from torch.utils.data import random_split
import torch.nn.functional as F
from torch.utils.data import DataLoader
from ReVel.perturbations import get_perturbation
from ReVel.load_data import load_data
from TSHIELD import TSHIELD
from TSHIELD.procedures import procedures

device = "cuda" if torch.cuda.is_available() else "cpu"
# Load the model
# Download the dataset Flowers and change the last layer to fit the number of classes
classifier = procedures.classifier("efficientnet-b2", num_classes=102)
perturbation = get_perturbation(name="square",dim=9,num_classes= 102,
    final_size=(224, 224),kernel=150.0,max_dist=20,ratio=0.5)

train_set = load_data("Flowers", perturbation=perturbation, train=True, dir="./data/")
test_set = load_data("Flowers", perturbation=perturbation, train=False, dir="./data/")
classifier.to(device)

Train, Val = random_split(
    train_set, [int(len(train_set) * 0.9), len(train_set) - int(len(train_set) * 0.9)]
)
# Subset of 5% of train and 5% of val

Train = torch.utils.data.Subset(Train, range(int(len(Train) * 0.05)))
Val = torch.utils.data.Subset(Val, range(int(len(Val) * 0.05)))

TrainLoader = DataLoader(Train, batch_size=32, shuffle=True)
ValLoader = DataLoader(Val, batch_size=32, shuffle=False)

def loss_f(y_pred,y_label):
    return F.cross_entropy(y_pred,torch.argmax(y_label,dim=1))
optimizer = torch.optim.AdamW(classifier.parameters(), lr=0.001,
    weight_decay=0.01, amsgrad=True)
epochs = 5 # Change the number of epochs in case you need more
best_loss = torch.tensor(float("inf"))

Training and validation phase
=============================

In [6]:
for epoch in range(epochs):
    print(f"Epoch :{epoch+1}, {(epoch+1)/epochs*100:.2f}%")
    train_loss, train_acc, train_reg = procedures.train_step(
        ds_loader=TrainLoader,
        model=classifier,
        optimizer=optimizer,
        loss_f=loss_f,
        reg_f=lambda x, y: (TSHIELD.rshield(model=x, input=y, percentage=5, device=device)),
        device=device,
    )
    print(f"Train Loss: {train_loss:.4f}, Train Accuracy: {train_acc:.4f}, Train Regularization: {train_reg:.4f}")
    val_loss, val_acc, val_reg = procedures.validation_step(
        ds_loader=ValLoader,
        model=classifier,
        loss_f=loss_f,
        reg_f=lambda x, y: (TSHIELD.rshield(model=x, input=y, percentage=5, device=device)),
        device=device,
    )
    print(f"Validation Loss: {val_loss:.4f}, Validation Accuracy: {val_acc:.4f}, Validation Regularization: {val_reg:.4f}")
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(classifier.state_dict(), "./model.pth")

Epoch :1, 20.00%


Training:   0%|          | 0/3 [00:00<?, ?it/s]/home/ivan/.local/lib/python3.12/site-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
Training: 100%|██████████| 3/3 [02:18<00:00, 46.13s/it, loss=0.165, acc=0.011, reg=0.0245] 


Train Loss: 0.1655, Train Accuracy: 0.0110, Train Regularization: 0.0245


Validation: 100%|██████████| 1/1 [00:05<00:00,  5.87s/it, loss=0.45, acc=0.1, reg=0.0151]


Validation Loss: 0.4502, Validation Accuracy: 0.1000, Validation Regularization: 0.0151
Epoch :2, 40.00%


...

Validation Loss: 0.4455, Validation Accuracy: 0.2000, Validation Regularization: 0.0119
Epoch :5, 100.00%


Training: 100%|██████████| 3/3 [02:18<00:00, 46.21s/it, loss=0.00826, acc=1, reg=0.0121]


Train Loss: 0.0083, Train Accuracy: 1.0000, Train Regularization: 0.0121


Validation: 100%|██████████| 1/1 [00:05<00:00,  5.89s/it, loss=0.446, acc=0.2, reg=0.0125]

Validation Loss: 0.4462, Validation Accuracy: 0.2000, Validation Regularization: 0.0125


Testing phase
=============

In [7]:
classifier.load_state_dict(torch.load("./model.pth"))
test = DataLoader(test_set, batch_size=32, shuffle=False)
test_loss, test_acc, test_reg = procedures.validation_step(
    ds_loader=test,
    model=classifier,
    loss_f=loss_f,
    reg_f=lambda x, y: (TSHIELD.rshield(model=x, input=y, percentage=5, device=device)),
    device=device,
)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_acc:.4f}, Test Regularization: {test_reg:.4f}")

/tmp/ipykernel_91007/2827991990.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  classifier.load_state_dict(torch.load("./model.pth"))
Validation: 100%|██████████| 193/19

Test Loss: 0.1247, Test Accuracy: 0.1978, Test Regularization: 0.0053
